# PL01

In [145]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [146]:
#dataset original
df = pd.read_csv('datasets/pagamentos_quotas.csv')

In [147]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60340 entries, 0 to 60339
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   pagamento_id      60340 non-null  object 
 1   fração_id         60340 non-null  object 
 2   mes_referencia    60340 non-null  object 
 3   data_vencimento   60340 non-null  object 
 4   data_pagamento    59588 non-null  object 
 5   valor_quota       60340 non-null  object 
 6   multa             59700 non-null  float64
 7   estado            60340 non-null  object 
 8   metodo_pagamento  59016 non-null  object 
dtypes: float64(1), object(8)
memory usage: 4.1+ MB


In [148]:
#dataset copia
df2 = df.copy()
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60340 entries, 0 to 60339
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   pagamento_id      60340 non-null  object 
 1   fração_id         60340 non-null  object 
 2   mes_referencia    60340 non-null  object 
 3   data_vencimento   60340 non-null  object 
 4   data_pagamento    59588 non-null  object 
 5   valor_quota       60340 non-null  object 
 6   multa             59700 non-null  float64
 7   estado            60340 non-null  object 
 8   metodo_pagamento  59016 non-null  object 
dtypes: float64(1), object(8)
memory usage: 4.1+ MB


In [149]:
#funçao para retirar espaços e caracteres nao numericos

def limpeza(valor):
    if pd.isna(valor):
        return ('Nan')
    valor = str(valor)
    valor = valor.upper().replace('EURO','').replace('€','')
    valor = valor.strip()
    valor = ''.join(c for c in valor if c.isdigit() or c in [',','.'])
    valor = valor.replace(',','.')
    return float(valor)

In [150]:
#conversao de tipo de dados
df2['valor_quota'] = df2['valor_quota'].apply(limpeza)
df2['pagamento_id'] = df2['pagamento_id'].astype('string')
df2['fração_id'] = df2['fração_id'].astype('string')
df2['mes_referencia'] = df2['mes_referencia'].astype('string')
df2['data_vencimento'] =pd.to_datetime(df['data_vencimento'],format='mixed')
df2['data_pagamento'] =pd.to_datetime(df['data_pagamento'],errors='coerce')
df2['valor_quota'] = df2['valor_quota'].astype('float')
df2['estado'] = df2['estado'].astype('string')
df2['metodo_pagamento'] = df2['metodo_pagamento'].astype('string')
#astype
df2.dtypes

pagamento_id        string[python]
fração_id           string[python]
mes_referencia      string[python]
data_vencimento     datetime64[ns]
data_pagamento      datetime64[ns]
valor_quota                float64
multa                      float64
estado              string[python]
metodo_pagamento    string[python]
dtype: object

In [167]:
#formaataçao de dados
df2["multa"] = df2["multa"].fillna(df2['multa'].mean())
df2["metodo_pagamento"] = df2["metodo_pagamento"].fillna(df2['metodo_pagamento'].mode()[0])
df2["status_pagamento"] = df2["data_pagamento"].apply(lambda x: "nao pago" if pd.isna(x) else "pago")




In [152]:
#verificaçao de valores nulos no dataset
df2.isnull().sum()

pagamento_id          0
fração_id             0
mes_referencia        0
data_vencimento       0
data_pagamento      825
valor_quota           0
multa                 0
estado                0
metodo_pagamento      0
status_pagamento      0
dtype: int64

In [153]:
#eliminar duplicados
df2 = df2.drop_duplicates()

In [154]:
#verificaçao de valores nulos no dataset
df2.isnull().sum()

pagamento_id          0
fração_id             0
mes_referencia        0
data_vencimento       0
data_pagamento      825
valor_quota           0
multa                 0
estado                0
metodo_pagamento      0
status_pagamento      0
dtype: int64

In [155]:
#formataçao dos dados da coluna metodo_pagamento
df2['metodo_pagamento'] = df2['metodo_pagamento'].str.lower().str.strip()
df2['metodo_pagamento'].value_counts()

metodo_pagamento
transferência    29746
mb way           20186
cheque            5964
numerário         2894
mbway              429
transf.            304
transfer           300
transferencia      275
chq                 72
cash                27
dinheiro            27
numerario           18
Name: count, dtype: Int64

In [156]:
metodos_pagamento = {
    'cartao': 'transferência',
    'transf.': 'transferência',
    'transfer': 'transferência',
    'transferencia': 'transferência',
    'mb way': 'MB WAY',
    'mbway': 'MB WAY',
    'cheque': 'cheque',
    'chq': 'cheque',
    'dinheiro': 'dinheiro',
    'numerário':'dinheiro',
    'numerario':'dinheiro',
    'cash':'dinheiro'
}
df2['metodo_pagamento'] = df2['metodo_pagamento'].replace(metodos_pagamento)
df2['metodo_pagamento'].value_counts()

metodo_pagamento
transferência    30625
MB WAY           20615
cheque            6036
dinheiro          2966
Name: count, dtype: Int64

In [157]:
#formataçao dos dados da coluna estado
df2['estado'] = df2['estado'].str.lower().str.strip()
df2['estado'].value_counts()

estado
pago         44808
em atraso    13679
pendente       752
paid           384
pg             374
atraso         128
atrasado       117
Name: count, dtype: Int64

In [158]:
metodos_pagamento = {
    'paid': 'pago',
    'paid': 'pago',
    'pg': 'pago',
    'em atraso': 'em atraso',
    'atraso': 'em atraso',
    'atrasado': 'em atraso',
    'pendente': 'em atraso'
}
df2['estado'] = df2['estado'].replace(metodos_pagamento)
df2['estado'].value_counts()

estado
pago         45566
em atraso    14676
Name: count, dtype: Int64

In [159]:
#Deteção de padrões nos dados através de agregações com .groupby();
#Cálculo de estatísticas descritivas segmentadas por grupos relevantes

#total de multas por metodo de pagamento
df2.groupby('metodo_pagamento')[['valor_quota','multa']].sum()

,valor_quota,multa
metodo_pagamento,,
MB WAY,3937829.21,69525.729787
cheque,1273560.21,21236.289934
dinheiro,598244.92,7695.639968
transferência,5858200.75,97588.329668


In [160]:
#numero de multas e a media de multas por estado
df2.groupby('estado').agg(
    total_pagamentos=('pagamento_id','count'),
    multa_media=('multa','mean'),
    valor_total=('valor_quota','sum')
)


,total_pagamentos,multa_media,valor_total
estado,,,
em atraso,14676,13.254411,2846661.23
pago,45566,0.033451,8821173.86


In [161]:

#dias de pagamento antes depois vencimento
df2['dias_atraso'] = (df2['data_pagamento']-df2['data_vencimento']).dt.days
df2.groupby('fração_id')['dias_atraso'].mean()

fração_id
FR0001     3.413043
FR0002     2.583333
FR0003     9.413043
FR0004     0.404255
FR0005     3.479167
            ...    
FR2496     3.541667
FR2497     0.916667
FR2498    15.739130
FR2499    19.739130
FR2500     1.916667
Name: dias_atraso, Length: 2500, dtype: float64

In [165]:
print(f'MEMORIA TOTAL OCUPADA PELO DATASET NAO OTIMIZADO : {round(df.memory_usage(deep=True).sum()/1024,2)}KB')

MEMORIA TOTAL OCUPADA PELO DATASET NAO OTIMIZADO : 28056.99KB


In [ ]:
print(f'MEMORIA TOTAL OCUPADA PELO DATASET OTIMIZADO : {round(df2.memory_usage(deep=True).sum()/1024,2)}KB')

MEMORIA TOTAL OCUPADA PELO DATASET  OTIMIZADO : 23395.65KB


In [162]:
df2.to_csv("datasets/pagamentos_clean.csv")